# Satellite Tracking Data
## Union of Concerned Scientists

Assembled by experts at the Union of Concerned Scientists (UCS), the [Satellite Database](https://www.ucs.org/resources/satellite-database) is a listing of the more than 7,560 operational satellites currently in orbit around Earth. It was first published on Dec 8, 2005 and most recently updated on May 1, 2023.

![Loose Ends, Long Goodbyes Sat](img/lelg_sat_small.png)

Much like orbital debris plummeting into the atmosphere, the dataset requires some cleanup. To begin, we'll import the necessary libraries, read the csv, and take a look at the total rows and columns:

In [ ]:
import pandas as pd
import numpy as np
import re
import json

file_path = "data/UCS-Satellite-Database 5-1-2023.csv"
df = pd.read_csv(file_path) # you can read a csv, parquet, json, etc. Break it into chunks if the dataset is huge. Specify encoding if needed.
df.shape # returns (rows, columns) in the df

(7562, 68)

### Basic Cleaning
#### Columns

Let's check out the names of the columns in the dataframe.

In [50]:
df.columns # prints names of all columns

Index(['Name of Satellite, Alternate Names',
       'Current Official Name of Satellite', 'Country/Org of UN Registry',
       'Country of Operator/Owner', 'Operator/Owner', 'Users', 'Purpose',
       'Detailed Purpose', 'Class of Orbit', 'Type of Orbit',
       'Longitude of GEO (degrees)', 'Perigee (km)', 'Apogee (km)',
       'Eccentricity', 'Inclination (degrees)', 'Period (minutes)',
       'Launch Mass (kg.)', ' Dry Mass (kg.) ', 'Power (watts)',
       'Date of Launch', 'Expected Lifetime (yrs.)', 'Contractor',
       'Country of Contractor', 'Launch Site', 'Launch Vehicle',
       'COSPAR Number', 'NORAD Number', 'Comments', 'Unnamed: 28',
       'Source Used for Orbital Data', 'Source', 'Source.1', 'Source.2',
       'Source.3', 'Source.4', 'Source.5', 'Source.6', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49',


Pandas names blank columns 'Unnamed: X'. Let's see how many we have.

In [51]:
unnamed_cols = [col for col in df.columns if col.startswith("Unnamed")] #substring match

print(f"This dataset has {len(unnamed_cols)} unnamed columns.")


This dataset has 32 unnamed columns.


Those can most likely be dropped, but let's make sure they are actually empty.

In [52]:
blank_cols = [
    col for col in df.columns
    if df[col].isna().all() or df[col].astype(str).str.strip().eq("").all() # remove whitespace (strip) then match for non-null but blank (eq)
]

blank_cols


[]

Mysterious. These aren't empty but I suspect they are an artifact and the data they contain isn't useful. Let's grab a couple and look at the value counts.

In [53]:
df["Unnamed: 37"].value_counts() # sum of values


Unnamed: 37
Estimated    484
Name: count, dtype: int64

We see only a single value: *Estimated*. While technically this isn't empty, it isn't anything informative either. Let's check how many columns in the dataset contain only a single value (not counting nulls or blanks).

In [54]:
single_value_cols = [
    col for col in df.columns
    if df[col]
        .replace(r"^\s*$", pd.NA, regex=True)
        .dropna()
        .nunique() == 1
]

print(f"There are {len(single_value_cols)} columns that contain only a single value.")

There are 27 columns that contain only a single value.


We could dive into this deeper and look at all 27, but that's tedious and not an optimal use of our limited time in this universe. Instead, this looks like a great place to add a constraint:
> If a column has only one value, it is not useful, and therefore we can drop it.

In [55]:
df = df.drop(columns=single_value_cols)
unnamed_cols = [col for col in df.columns if col.startswith("Unnamed")]

print(f"This dataset has {len(unnamed_cols)} unnamed columns.")

This dataset has 5 unnamed columns.


That got rid of most of the `Unnamed` columns. While those remaining 5 have multiple values, at this point it's reasonable to assume they aren't useful. Likewise, the `Source` and `Comments` columns aren't relevant for our analysis, and the `Name of Satellite, Alternate Names` is redundant. We can drop those:

In [56]:
original_cols = len(df.columns)
columns_to_drop = []

for col in df.columns:
    if col.startswith("Unnamed"):
        columns_to_drop.append(col) # adds column
    elif col.startswith("Source"):
        columns_to_drop.append(col)
    elif col == "Comments":
        columns_to_drop.append(col)
    elif col == "Name of Satellite, Alternate Names":
        columns_to_drop.append(col)

df = df.drop(columns=columns_to_drop)

dropped_cols = len(columns_to_drop)
remaining_cols = len(df.columns)

print(f"Dropped {dropped_cols} out of {original_cols} columns.")
print(f"Remaining columns: {remaining_cols}")


Dropped 15 out of 41 columns.
Remaining columns: 26


We're still focused on columns, but let's take a quick detour to verify how many rows are in the dataset. This figure will be useful as a point of reference.

In [57]:
total_rows = len(df)
print(f"The dataset has {total_rows} total rows.")

The dataset has 7562 total rows.


We'll check which columns are missing the most values. This helps to decide which columns are useful and which may be too incomplete.

In [58]:
missing_values = df.isna().sum() # map column name to count of null values
missing_values.sort_values(ascending=False)

Power (watts)                         6983
 Dry Mass (kg.)                       6795
Detailed Purpose                      6308
Expected Lifetime (yrs.)              2112
Type of Orbit                          653
Launch Mass (kg.)                      247
Period (minutes)                        58
Eccentricity                            13
Apogee (km)                              9
Perigee (km)                             9
Inclination (degrees)                    6
Longitude of GEO (degrees)               5
Country/Org of UN Registry               3
Date of Launch                           3
Contractor                               2
Country of Contractor                    2
Launch Site                              2
Launch Vehicle                           2
COSPAR Number                            2
Current Official Name of Satellite       2
Class of Orbit                           2
Purpose                                  2
Users                                    2
Operator/Ow

Let's look at that as a percentage to make the comparison easier:

In [59]:
missing_percent = (missing_values / total_rows) * 100
missing_percent_sorted = missing_percent.sort_values(ascending=False)
missing_percent_sorted.map(lambda x: f"{x:.1f}%")


Power (watts)                         92.3%
 Dry Mass (kg.)                       89.9%
Detailed Purpose                      83.4%
Expected Lifetime (yrs.)              27.9%
Type of Orbit                          8.6%
Launch Mass (kg.)                      3.3%
Period (minutes)                       0.8%
Eccentricity                           0.2%
Apogee (km)                            0.1%
Perigee (km)                           0.1%
Inclination (degrees)                  0.1%
Longitude of GEO (degrees)             0.1%
Country/Org of UN Registry             0.0%
Date of Launch                         0.0%
Contractor                             0.0%
Country of Contractor                  0.0%
Launch Site                            0.0%
Launch Vehicle                         0.0%
COSPAR Number                          0.0%
Current Official Name of Satellite     0.0%
Class of Orbit                         0.0%
Purpose                                0.0%
Users                           

If a column is missing data in 25% or more of the total rows, we will exclude it:

In [60]:
missing_threshold = total_rows // 4

columns_to_exclude = missing_values[missing_values >= missing_threshold].index.tolist() # indexed on column name
df = df.drop(columns=columns_to_exclude) # 1-dimensional array is a Series in Pandas.
print(f"Dropped {len(columns_to_exclude)} columns: {(', '.join(columns_to_exclude))}")


Dropped 4 columns: Detailed Purpose,  Dry Mass (kg.) , Power (watts), Expected Lifetime (yrs.)


We're almost done with the column cleanup. As a final touch, let's rename some of the more verbose columns. We'll also drop any units of measure contained in the column names (such as km).

In [61]:
column_renames = {
    "Current Official Name of Satellite": "Satellite",
    "Country/Org of UN Registry": "UN Registry",
    "Country of Operator/Owner": "Country of Operator",
    "Operator/Owner": "Operator"
}

df = df.rename(columns=column_renames)

df.columns = (
    df.columns
        .str.replace(r"\s*\(.*?\)", "", regex=True)
        .str.strip()
)

for col in df.columns:
    print(col)

Satellite
UN Registry
Country of Operator
Operator
Users
Purpose
Class of Orbit
Type of Orbit
Longitude of GEO
Perigee
Apogee
Eccentricity
Inclination
Period
Launch Mass
Date of Launch
Contractor
Country of Contractor
Launch Site
Launch Vehicle
COSPAR Number
NORAD Number


#### Rows
We can tolerate missing data in certain fields, but others are essential for the analysis. If any rows have data missing data in these fields, they will either need to be dropped entirely or fixed. Let's take a look.

In [62]:
required_columns = [
    "Date of Launch",
    "Launch Site",
    "Satellite"
]

rows_missing_required = df[
    df[required_columns].isna().any(axis=1)
][required_columns]

rows_missing_required

,Date of Launch,Launch Site,Satellite
240,NaN,Rocket Lab Launch Complex 1,BlackSky Global 5
7560,NaN,NaN,NaN
7561,NaN,NaN,NaN


The satellite **BlackSky Global 5** is missing a launch date. However, this [information](https://space.oscar.wmo.int/satellites/view/blacksky_5) isn't too hard to find so let's just manually add it in: August 7, 2020.

The other two rows are just blanks beyond redemption, so let's delete them.

In [63]:
rows_before = len(df)

df.loc[
    df["Satellite"] == "BlackSky Global 5",
    "Date of Launch"
] = "2020-08-07"

rows_missing_required = df[
    df[required_columns].isna().any(axis=1)
]

df = df.drop(index=rows_missing_required.index)

rows_after = len(df)
rows_removed = rows_before - rows_after

print(f"Dropped {rows_removed} out of {rows_before} rows.")
print(f"Remaining rows: {rows_after}")

Dropped 2 out of 7562 rows.
Remaining rows: 7560


Many of the text columns contain extra information in parenthesis. Let's look at the `Satellite` column as an example.

In [ ]:
mask_parens = (
    df["Satellite"]
        .astype(str)
        .str.contains(r"\(", na=False)
)

df.loc[mask_parens, ["Satellite"]].head(5)

We'll just clean house and remove these parenthetical notes and trim whitespace while we're at it. We'll do this for all columns.

In [ ]:
text_cols = df.select_dtypes(include=["object"]).columns.tolist()

for col in text_cols:
    df[col] = (
        df[col]
            .str.replace(r"\s*\(.*?\)", "", regex=True)
            .str.strip()
            .replace({"": pd.NA})
    )

df.loc[mask_parens, ["Satellite"]].head(5)

### Making Booleans
#### `Users`

Now that we've got the basic dataset cleaned up, we'll take a look at some of the specific columns.

`Users` contains combinations of one to four possible values separated by a "/", representing who is using the Satellite: *Civil*, *Commercial*, *Government*, or *Military*. This format is easy enough for a human to read in a table, but a one-to-many relationship like this will be complicated to filter in a BI tool like QuickSight or Tableau. Instead, let's make life easier by creating some boolean columns. 

In [ ]:
user_categories = ["Civil", "Commercial", "Government", "Military"]

for category in user_categories:
    df[f"User: Is {category}"] = (
        df["Users"]
            .str.contains(category, case=False, na=False)
    )

user_flag_columns = ["User: Is Civil", "User: Is Commercial", "User: Is Government", "User: Is Military"]

df[user_flag_columns].sum()

example_columns = [
    "Satellite",
    "Users",
    "User: Is Civil",
    "User: Is Commercial",
    "User: Is Government",
    "User: Is Military"
]

df[
    df[user_flag_columns].sum(axis=1) > 1
][example_columns].head(10)



#### `Purpose`

We'll do the same thing for `Purpose`. However, here there are a few more categories. If we make bools for everything, this could get clunky. As an intermediate step though, it works fine.

In [ ]:

purpose_categories = [
    "Communications",
    "Earth Observation",
    "Earth Science",
    "Educational",
    "Meteorological",
    "Mission Extension Technology",
    "Navigation",
    "Platform",
    "Satellite Positioning",
    "Space Observation",
    "Space Science",
    "Surveillance",
    "Technology Demonstration",
    "Technology Development",
    "Unknown",
    "Maritime Tracking"
]

for category in purpose_categories:
    df[f"Is Purpose: {category}"] = (
        df["Purpose"]
            .str.contains(category, case=False, na=False)
    )

The count by category varies significantly. Some have thousands while others only several.

In [ ]:
purpose_flag_cols = [f"Is Purpose: {c}" for c in purpose_categories]

df[purpose_flag_cols].sum().sort_values(ascending=False)


In a lopsided situation like this, booleans become less effective. Let's consolidate some of these categories for better clarity.

In [ ]:
p_comm = "Is Purpose: Communications"
p_earth_obs = "Is Purpose: Earth Observation"
p_earth_sci = "Is Purpose: Earth Science"
p_edu = "Is Purpose: Educational"
p_met = "Is Purpose: Meteorological"
p_mext = "Is Purpose: Mission Extension Technology"
p_nav = "Is Purpose: Navigation"
p_platform = "Is Purpose: Platform"
p_satpos = "Is Purpose: Satellite Positioning"
p_space_obs = "Is Purpose: Space Observation"
p_space_sci = "Is Purpose: Space Science"
p_surv = "Is Purpose: Surveillance"
p_tech_demo = "Is Purpose: Technology Demonstration"
p_tech_dev = "Is Purpose: Technology Development"
p_unknown = "Is Purpose: Unknown"
p_mar = "Is Purpose: Maritime Tracking"


df["Purpose: Communications"] = df[p_comm]

df["Purpose: Earth Observation"] = (
    df[p_earth_obs]
    | df[p_met]
    | df[p_surv]
    | df[p_earth_sci]
    | df[p_mar]
)

df["Purpose: Navigation"] = (
    df[p_nav]
    | df[p_satpos]
)

df["Purpose: Space Science"] = (
    df[p_space_sci]
    | df[p_space_obs]
)

df["Purpose: Tech Dev"] = (
    df[p_tech_dev]
    | df[p_edu] 
    | df[p_platform] 
    | df[p_tech_demo]
    | df[p_mext]
)

df["Purpose: Unknown"] = df[p_unknown]


This gets us down to six `Purpose` boolean columns which will be much more manageable.

In [ ]:
consolidated_cols = [
    "Purpose: Communications",
    "Purpose: Earth Observation",
    "Purpose: Navigation",
    "Purpose: Space Science",
    "Purpose: Tech Dev",
    "Purpose: Unknown",
]

original_purpose_flag_cols = [c for c in df.columns if c.startswith("Is Purpose: ")]
df = df.drop(columns=original_purpose_flag_cols)

df[consolidated_cols].sum().sort_values(ascending=False)

### Aggregating
#### `Type of Orbit` and `Class of Orbit`

Next let's take a look the type of orbit. Unlike `Users` or `Purpose`, `Type of Orbit` is a 1:1 relationship. A satellite will only have a single type of orbit. However, this also looks like it could benefit from some consolidation.

In [ ]:
orbit_counts = (
    df["Type of Orbit"]
        .value_counts()
        .sort_values(ascending=False)
)

orbit_counts

Since some of these orbits could be considered [subtypes](https://en.wikipedia.org/wiki/Orbit), we'll aggregate them into six larger categories. The new column we will consider to be the broader `Orbit Category`.

In [ ]:
orbit_mapping = {
    "Non-Polar Inclined": "Inclined",
    "Sun-Synchronous": "Sun-Synchronous",
    "Sun-Synchronous near polar": "Sun-Synchronous",
    "Polar": "Polar",
    "Equatorial": "Equatorial",
    "Molniya": "Highly Elliptical",
    "Deep Highly Eccentric": "Highly Elliptical",
    "Elliptical": "Highly Elliptical",
    "Retrograde": "Other",
    "Cislunar": "Other",
}

df["Orbit Category"] = (
    df["Type of Orbit"].map(orbit_mapping) # map replaces one value with another
)

df["Orbit Category"].value_counts()


The `Class of Orbit` refers to the altitude of the orbit. The original dataset is divided into two broad classes: 
* *nearly circular orbits*: LEO, MEO, and GEO
* *elliptical orbits*
Satellites in elliptical orbits have apogees and perigees that differ significantly from each other. They spend time at many different altitudes above the earth’s surface.

In [ ]:
df["Class of Orbit"].value_counts()

However, for the purposes of our analysis, we'll consider elliptical orbits to be *High Earth Orbits (HEO)* based on the apogee. This will avoid confusion the category in `Type of Orbit`. We'll also clean up the capitalization type for Low Earth Orbit.

In [ ]:

df["Class of Orbit"] = (
    df["Class of Orbit"]
        .replace({
            "LEo": "LEO",
            "Elliptical": "HEO"
        })
)

df["Class of Orbit"].value_counts()


Since the shape of the orbit is still relevant though, we'll derive this from `Class of Orbit` and capture this in a new field: `Shape of Orbit`.

In [ ]:
df["Shape of Orbit"] = (
    df["Class of Orbit"]
        .map({
            "LEO": "Circular",
            "MEO": "Circular",
            "GEO": "Circular",
            "HEO": "Elliptical",
        })
)

df["Shape of Orbit"].value_counts()

Now that we've got three fields of orbit data, let's do another rename so they are easier to find later in our BI tool. We'll make sure everything starts with *Orbit*. We'll also rename `Type of Orbit` to `Orbit Subcategory` to be better reflect the relationship.

In [ ]:
df = df.rename(columns={
    "Type of Orbit": "Orbit Subcategory",
    "Class of Orbit": "Orbit Class",
    "Shape of Orbit": "Orbit Shape",
})

[c for c in df.columns if "Orbit" in c]

### Mapping and Normalizing
#### `Operator` and `Contractor`

Here's where things get messy. The fields of `Operator` and `Contractor` were manually entered and cover a wide variety of organizations. As a result, several distinct issues appear:

* **Minor formatting and naming variations**: The same organization may appear multiple times due to differences in capitalization, punctuation, spacing, or legal suffixes

* **Parent companies vs. subsidiaries or internal divisions**: Some organizations are listed as specific subsidiaries, regional branches, or internal divisions of a larger organization

* **Joint or multi-organization missions**: Certain satellites are operated jointly by multiple organizations and are recorded as a combined value

* **Mixed organization types**: This includes private companies, government agencies, military organizations, universities, and research institutions that do not easily fit into a single hierarchy

For our purposes, granular distinction is beyond the scope of this project. We will do what we can do simplify.

In [ ]:
op_sample = (
    pd.Series(df["Operator"].dropna().unique())
      .sample(5, random_state=42)
      .tolist()
)

total_op_begin = df["Operator"].nunique()
total_con_begin = df["Contractor"].nunique()

print(f"Operators: {total_op_begin}")
print(f"Contractors: {total_con_begin}")
print(f"Sample: {', '.join(op_sample)}")

That's a fairly large amount to clean up. To begin, let's see what we can do with fixing the formatting. We'll remove suffices, trim whitespace, and apply other normalization standards.

In [ ]:
def normalize_org_column(df, col_name):
    s = (
        df[col_name]
            .astype(str)
            .str.strip()
            .str.replace(r"\s+", " ", regex=True)                 # collapse multiple spaces
            .str.replace(r"\s*/\s*", "/", regex=True)             # normalize slash spacing
            .str.replace(r",", "", regex=True)                    # remove commas
            .str.replace(r"\s+\.", ".", regex=True)               # remove space before period
            .str.replace(r"\.$", "", regex=True)                  # drop trailing periods
            .str.replace(
                r"\b(Ltd|Inc|LLC|PLC|Corp|Corporation|Co|S A|SA)\b",
                "",
                regex=True
            )                                                     # drop common suffixes
            .str.replace(r"\s+", " ", regex=True)                 # re-collapse spaces
            .str.strip()
            .replace({"nan": pd.NA})
    )
    
    s = s.str.split("/").str[0].str.strip() # Keep first entry

    df[col_name] = s
    return df[col_name]

normalize_org_column(df, "Operator")
normalize_org_column(df, "Contractor")

op_sample_2 = (
    pd.Series(df["Operator"].dropna().unique())
      .sample(5, random_state=42)
      .tolist()
)

total_op_begin_2 = df["Operator"].nunique()
total_con_begin_2 = df["Contractor"].nunique()

print(f"Operators: {total_op_begin_2} (consolidated {total_op_begin - total_op_begin_2})")
print(f"Contractors: {total_con_begin_2} (consolidated {total_con_begin - total_con_begin_2})")
print(f"Sample: {', '.join(op_sample_2)}")

From trial and error, I found that universities were particularly challenging to cleanup. Here are some functions specific for academic institutions.

In [ ]:
def safe_university_cleanup(s: str):
    if s is None or pd.isna(s):
        return s

    s = str(s).strip()

    if "University" not in s:
        return s

    m = re.search(r"\bUniversity\b.*", s) # Grab from the first "University" onward
    if not m:
        return s

    uni_part = m.group(0)
    uni_part = re.split(r"[,/]", uni_part, maxsplit=1)[0].strip()

    if uni_part.lower() == "university": # Never collapse to just "University"
        return s

    return uni_part


def strip_trailing_space_and_punct(series: pd.Series) -> pd.Series:
    return (
        series
            .astype("string")
            .str.replace(r"[\s\.\,\;\:\-]+$", "", regex=True)
            .str.strip()
            .replace({"nan": pd.NA})
    )


def apply_academic_and_trailing_cleanup(df, col_name):
    df[col_name] = df[col_name].apply(safe_university_cleanup)
    df[col_name] = strip_trailing_space_and_punct(df[col_name])
    return df[col_name]

apply_academic_and_trailing_cleanup(df, "Operator")
apply_academic_and_trailing_cleanup(df, "Contractor")

total_op_begin_3 = df["Operator"].nunique()
total_con_begin_3 = df["Contractor"].nunique()

print(f"Operators: {total_op_begin_3} (consolidated {total_op_begin_2 - total_op_begin_3})")
print(f"Contractors: {total_con_begin_3} (consolidated {total_con_begin_2 - total_con_begin_3})")

Now that the basic cleanup is done, on to the big step: mapping. Since there are over 300 organizations that need mapping, I've put these into a separate JSON we'll be loading in. Identifying the corrections took a fairly substantial amount of manual effort and required some judgement calls as well. This task would be a good candidate for ML, but that's beyond the scope of this project.

In [ ]:
with open("data/op_con_mapping.json", "r") as f:
    op_con_mapping = json.load(f)

def apply_canonical_mapping(df, col_name: str, mapping: dict):

    df[col_name] = df[col_name].map(mapping).fillna(df[col_name]) # Fallback to existing name
    df[col_name] = df[col_name].fillna("Unknown") # fill missing with "Unknown"

    return df[col_name]

apply_canonical_mapping(df, "Operator", op_con_mapping)
apply_canonical_mapping(df, "Contractor", op_con_mapping)

total_op_begin_4 = df["Operator"].nunique()
total_con_begin_4 = df["Contractor"].nunique()

print(f"Operators: {total_op_begin_4} (consolidated {total_op_begin_3 - total_op_begin_4})")
print(f"Contractors: {total_con_begin_4} (consolidated {total_con_begin_3 - total_con_begin_4})")

That narrows it down a bit. Let's take a peak at the Top 10 most common Operators and Contractors.

In [ ]:
print("Top 10 Operators:")
display(df["Operator"].value_counts().head(10))

print("Top 10 Contractors:")
display(df["Contractor"].value_counts().head(10))


#### `Launch Vehicle`

There's a lot more crammed into this field than the name suggests. Some are actual launch vehicles while other include (or substitute) the launch method, the booster stage, or the spacecraft.

In [ ]:
lv_count = df["Launch Vehicle"].nunique()

lv_sample = (
    pd.Series(df["Launch Vehicle"].dropna().unique())
      .sample(5, random_state=42)
      .tolist()
)

print(f"Launch Vehicles: {lv_count}")
print(f"Sample: {', '.join(lv_sample)}")

We'll begin by deriving the `Launch Vehicle Family`. This is the high-level rocket family for grouping and charts. We'll also make note of alternative methods for a later calculation.

Instead of using a dictionary, since the launch vehicles are more standardized, we'll use a substring search to do the mapping.

In [ ]:

def get_launch_vehicle_family(lv):
    if pd.isna(lv):
        return "Unknown"

    lv = lv.strip()

    if "Nanorack" in lv:
        return "Nanorack"
    if "Slingshot" in lv or "Dispenser" in lv:
        return "Deployer"

    if lv in {"L1011", "LauncherOne"}:
        return "Air Launch"

    if lv == "Space Shuttle":
        return "Space Shuttle"

    # Rocket families
    if lv.startswith("Falcon"):
        return "Falcon"
    if lv.startswith("Atlas"):
        return "Atlas"
    if lv.startswith("Soyuz"):
        return "Soyuz"
    if lv.startswith("Ariane"):
        return "Ariane"
    if lv.startswith("Long March"):
        return "Long March"
    if lv.startswith("PSLV"):
        return "PSLV"
    if lv.startswith("GSLV") or lv.startswith("LVM3"):
        return "GSLV"
    if lv.startswith("SSLV"):
        return "SSLV"
    if lv.startswith("Delta"):
        return "Delta"
    if lv.startswith("Titan"):
        return "Titan"
    if lv.startswith("Zenit"):
        return "Zenit"
    if lv.startswith("Minotaur"):
        return "Minotaur"
    if lv.startswith("Electron"):
        return "Electron"
    if lv.startswith("Antares"):
        return "Antares"
    if lv.startswith("Vega"):
        return "Vega"
    if lv.startswith("Pegasus"):
        return "Pegasus"
    if lv.startswith("Taurus"):
        return "Taurus"
    if lv.startswith("Proton"):
        return "Proton"
    if lv.startswith("Dnepr"):
        return "Dnepr"
    if lv.startswith("Nuri"):
        return "Nuri"
    if lv.startswith("Shavit"):
        return "Shavit"
    if lv.startswith("H2"):
        return "H-II"
    if lv.startswith("Kuaizhou"):
        return "Kuaizhou"
    if lv.startswith("Ceres"):
        return "Ceres"
    if lv.startswith("Rokot"):
        return "Rokot"
    if lv.startswith("Start"):
        return "Start"
    if lv.startswith("Qased"):
        return "Qased"
    if lv.startswith("Naro"):
        return "Naro"
    if lv.startswith("Jielong"):
        return "Jielong"
    if lv.startswith("Tsyklon"):
        return "Tsyklon"
    if lv.startswith("Rocket 3"):
        return "Rocket 3"
    if lv.startswith("Kosmos"):
        return "Kosmos"
    if lv.startswith("Epsilon"):
        return "Epsilon"
    if lv.startswith("JAXA M"):
        return "JAXA M-V"
    if lv.startswith("KT-"):
        return "KT-2"

    return "Other"

df["Launch Vehicle Family"] = df["Launch Vehicle"].apply(get_launch_vehicle_family)
df["Launch Vehicle Family"].value_counts()


Next, we'll explore the `Launch Method` to look at how the satellite reached orbit.

In [ ]:
def get_launch_method(lv):
    if pd.isna(lv):
        return "Unknown"

    if "Deployer" in lv or "Dispenser" in lv or "Slingshot" in lv:
        return "Deployer / Hosted Payload"

    if lv in {"L1011", "LauncherOne"}:
        return "Air Launch"

    if lv == "Space Shuttle":
        return "Space Shuttle"

    return "Orbital Rocket"

df["Launch Method"] = df["Launch Vehicle"].apply(get_launch_method)
df["Launch Method"].value_counts()

Since our mapping was just based on substrings, it's entirely possible that our initial pass missed some categorizations. Let's see if we can find any such anomalies.

In [ ]:
df[
    df["Launch Vehicle Family"].isin(["Other", "Deployer"])
][["Launch Vehicle", "Launch Vehicle Family", "Launch Method"]].head(20)


The *Other* category seems like it could benefit from further cleanup.

In [ ]:
other_lv = df[
    df["Launch Vehicle Family"] == "Other"
]["Launch Vehicle"]

print(f"Total records classified as 'Other': {len(other_lv)}")
print(f"\nLaunch Vehicle values classified as 'Other' (with counts):")

other_lv.value_counts()


We'll do another cleanup pass to reclassify "Other" launch vehicle families. Some values classified as "Other" are not actually launch vehicles such as upper stages or spacecraft. Since the atual launch vehicle wasn't provided, we can more accurately classify these as "Unknown".


In [ ]:
invalid_launch_vehicle_values = {
    "Breeze M",
    "Breeze KM",
    "Cygnus",
    "Fa",
}

mask_invalid_other = (
    (df["Launch Vehicle Family"] == "Other") &
    (df["Launch Vehicle"].isin(invalid_launch_vehicle_values))
)

df.loc[mask_invalid_other, "Launch Vehicle Family"] = "Unknown"

df["Launch Vehicle Family"].value_counts()

### Handling Dates
#### `Date of Launch`

To begin, let's see if any dates are incorrectly formatted.

In [ ]:
parsed_attempt = pd.to_datetime(
    df["Date of Launch"],
    errors="coerce",
    format="mixed"
)

bad_dates = df.loc[
    parsed_attempt.isna() & df["Date of Launch"].notna(),
    ["Satellite", "Date of Launch"]
].copy()

print("Rows with unparseable Date of Launch values:")
display(bad_dates)


We've got two. These look like basic typos so we can manually clean them up. Afterward, it'll be safe to parse the column as a datetime object.

In [ ]:
manual_date_fixes = {
    "Cicero-8": "11/29/2018",
    "Tianmu-1 01": "1/9/2023",
}

for sat, corrected in manual_date_fixes.items():
    df.loc[df["Satellite"] == sat, "Date of Launch"] = corrected

df["Date of Launch"] = pd.to_datetime(
    df["Date of Launch"],
    errors="coerce",
    format="mixed"
)

remaining_missing = df["Date of Launch"].isna().sum()
print(f"Remaining missing/unparseable Date of Launch values after fixes: {remaining_missing}")

display(df[df["Satellite"].isin(manual_date_fixes.keys())][
    ["Satellite", "Date of Launch"]
])

The dataset was last updated on May 1, 2023. Let's use this date to calculate the years in orbit.

In [ ]:
REFERENCE_DATE = pd.Timestamp("2023-05-01")

launch_date = pd.to_datetime(
    df["Date of Launch"],
    errors="coerce"
)

years_in_orbit = (
    (REFERENCE_DATE - launch_date).dt.days / 365.25
)

df["Years in Orbit"] = np.floor(years_in_orbit).astype("Int64")

df[["Satellite", "Date of Launch", "Years in Orbit"]].tail()


### Extrapolating Geographic Data
#### `Launch Site`

We've got launch sites, but we can use this to get a lot more geodata. To start though, the site names could use some basic cleanup.

In [ ]:
site_count = df["Launch Site"].dropna().nunique()
launch_sites = sorted(df["Launch Site"].dropna().unique())

print(f"Launch Sites: {site_count}")
for i in range(0, len(launch_sites), 5):
    print(", ".join(launch_sites[i:i+5]))

There's some variations on spelling (Center vs Centre) as well as some sites with additional information like the specific launch pad. We'll normalize this then also assign categorize the launch site. 

Specifically, we want to flag sites that are traditional spaceports since they can be flagged with geodata, as opposed to sea, air, and space-based launch platforms.

In [ ]:
def normalize_launch_site(site):
    if pd.isna(site):
        return ("Unknown", "Unknown")

    s = str(site).strip()

    # Space-based
    if "International Space Station" in s:
        return ("International Space Station", "Space-Based")

    # Air launch
    if s in {"Orbital ATK L-1011", "Stargazer L-1011", "Virgin Orbit"}:
        return ("Air Launch", "Air Launch")

    # Sea launch
    if s in {"Sea Launch Odyssey", "Yellow Sea Launch Platform"}:
        return ("Sea Launch", "Sea Launch")

    # Clearly not a site (wrong field)
    if s in {"Antares", "Cygnus", "FANTM-RAiL", "FANTM-RAiL [Xtenti]"}:
        return ("Unknown", "Unknown")

    # Geographic launch sites
    site_mapping = {
        "Cape Canaveral": "Cape Canaveral Space Force Station",
        "Vandeberg AFB": "Vandenberg Space Force Base",
        "Vandenberg AFB": "Vandenberg Space Force Base",
        "Baikonur Cosmodrome": "Baikonur Cosmodrome",
        "Guiana Space Center": "Guiana Space Center",
        "Jiuquan Satellite Launch Center": "Jiuquan Satellite Launch Center",
        "Xichang Satellite Launch Center": "Xichang Satellite Launch Center",
        "Taiyan Launch Center": "Taiyuan Launch Center",
        "Taiyuan Launch Center": "Taiyuan Launch Center",
        "Wenchang Satellite Launch Center": "Wenchang Space Center",
        "Wenchang Space Center": "Wenchang Space Center",
        "Plesetsk Cosmodrome": "Plesetsk Cosmodrome",
        "Vostochny Cosmodrome": "Vostochny Cosmodrome",
        "Svobodny Cosmodrome": "Svobodny Cosmodrome",
        "Satish Dhawan Space Centre": "Satish Dhawan Space Center",
        "Satish Dhawan Space Center": "Satish Dhawan Space Center",
        "Tanegashima Space Center": "Tanegashima Space Center",
        "Uchinoura Space Center": "Uchinoura Space Center",
        "Naro Space Center": "Naro Space Center",
        "Wallops Island Flight Facility": "Wallops Flight Facility",
        "Mid-Atlantic Regional Spaceport/Wallops Island": "Wallops Flight Facility",
        "Kodiak Island": "Pacific Spaceport Complex – Alaska",
        "Kodiak Launch Complex": "Pacific Spaceport Complex – Alaska",
        "Kwajalein Island": "Reagan Test Site (Kwajalein)",
        "Shahroud Missile Range": "Shahroud Missile Range",
        "Rocket Lab Launch Complex 1": "Rocket Lab Launch Complex",
        "Rocket Lab Launch Complex 1B": "Rocket Lab Launch Complex",
    }

    normalized = site_mapping.get(s, s)
    return (normalized, "Geographic")


# Apply normalization to create/overwrite Launch Site and Launch Site Type
df[["Launch Site", "Launch Site Type"]] = (
    df["Launch Site"].apply(lambda x: pd.Series(normalize_launch_site(x)))
)

# Quick check
df["Launch Site Type"].value_counts()

In [ ]:
# Print unique Launch Site values (sorted)
for site in sorted(df["Launch Site"].dropna().unique()):
    print(repr(site))

With the site names normalized, now we can add geodata for the Country, Region/State, and City of the launch site.

In [ ]:
launch_site_geo = {
    "Baikonur Cosmodrome": ("Kazakhstan", "Baikonur Region", "Baikonur"),
    "Cape Canaveral Space Force Station": ("United States", "Florida", "Cape Canaveral"),
    "Dombarovsky Air Base": ("Russia", "Orenburg Oblast", "Yasny"),
    "Guiana Space Center": ("France", "French Guiana", "Kourou"),
    "Jiuquan Satellite Launch Center": ("China", "Gansu", "Jiuquan"),
    "Naro Space Center": ("South Korea", "South Jeolla", "Goheung"),
    "Pacific Spaceport Complex – Alaska": ("United States", "Alaska", "Kodiak"),
    "Palmachim Launch Complex": ("Israel", "Central District", "Palmachim"),
    "Plesetsk Cosmodrome": ("Russia", "Arkhangelsk Oblast", "Mirny"),
    "Reagan Test Site (Kwajalein)": ("Marshall Islands", "Kwajalein Atoll", "Kwajalein"),
    "Rocket Lab Launch Complex": ("New Zealand", "Hawke’s Bay", "Mahia"),
    "Satish Dhawan Space Center": ("India", "Andhra Pradesh", "Sriharikota"),
    "Shahroud Missile Range": ("Iran", "Semnan Province", "Shahroud"),
    "Svobodny Cosmodrome": ("Russia", "Amur Oblast", "Svobodny"),
    "Taiyuan Launch Center": ("China", "Shanxi", "Taiyuan"),
    "Tanegashima Space Center": ("Japan", "Kagoshima Prefecture", "Minamitane"),
    "Uchinoura Space Center": ("Japan", "Kagoshima Prefecture", "Kimotsuki"),
    "Vandenberg Space Force Base": ("United States", "California", "Lompoc"),
    "Vostochny Cosmodrome": ("Russia", "Amur Oblast", "Tsiolkovsky"),
    "Wallops Flight Facility": ("United States", "Virginia", "Wallops Island"),
    "Wenchang Space Center": ("China", "Hainan", "Wenchang"),
    "Xichang Satellite Launch Center": ("China", "Sichuan", "Xichang"),
}

df[["Launch Country", "Launch Region", "Launch City"]] = (
    df["Launch Site"]
        .map(lambda s: launch_site_geo.get(s, (pd.NA, pd.NA, pd.NA)))
        .apply(pd.Series)
)


To confirm our geographic efforts have been successful, let's look at `Launch Site` counts for sites that have no country associated. These should only be air, sea, space, or unknown launch locations.

In [ ]:
df[
    df["Launch Country"].isna()
]["Launch Site"].value_counts()

We can also get a sneak peak at which sites have the most launches.

In [ ]:
launch_location_counts = (
    df
        .groupby(
            ["Launch Site", "Launch Country", "Launch Region", "Launch City"],
            dropna=False
        )
        .size()
        .reset_index(name="Count")
        .sort_values("Count", ascending=False)
)

launch_location_counts


Next, let's add latitude and longitude coordinates for the sites. We can use this for mapping in a visual and also use it for calculations. We'll check to make sure that all of our terrestrial sites have coordinates.

In [ ]:
launch_site_coords = {
    # Non-geographic / mobile
    "Air Launch": (pd.NA, pd.NA),
    "Sea Launch": (pd.NA, pd.NA),
    "International Space Station": (pd.NA, pd.NA),
    "Unknown": (pd.NA, pd.NA),

    # French Guiana
    "Guiana Space Center": (5.2360, -52.7750),

    # US
    "Vandenberg Space Force Base": (34.73734, -120.58431),
    "Wallops Flight Facility": (37.9402, -75.4664),
    "Pacific Spaceport Complex – Alaska": (57.44283, -152.35811),
    "Cape Canaveral Space Force Station": (28.48731, -80.57429),

    # Marshall Islands
    "Reagan Test Site (Kwajalein)": (8.72512, 167.72818),

    # Russia / Kazakhstan
    "Vostochny Cosmodrome": (51.8167, 128.2500),
    "Plesetsk Cosmodrome": (62.92805, 40.57559),
    "Dombarovsky Air Base": (51.09393, 59.84266),
    "Svobodny Cosmodrome": (51.88888, 128.11187),
    "Baikonur Cosmodrome": (45.9200, 63.3420),

    # China
    "Jiuquan Satellite Launch Center": (40.9546639, 100.2883333),
    "Taiyuan Launch Center": (38.8427806, 111.6050972), 
    "Xichang Satellite Launch Center": (28.2409417, 102.0226000),  
    "Wenchang Space Center": (19.6144917, 110.9511333), 

    # Japan
    "Tanegashima Space Center": (30.40000, 130.97000), 
    "Uchinoura Space Center": (31.25151, 131.07549),

    # India
    "Satish Dhawan Space Center": (13.719939, 80.230425), 

    # South Korea
    "Naro Space Center": (34.4319, 127.5351),

    # Israel
    "Palmachim Launch Complex": (31.89778, 34.69056), 

    # New Zealand
    "Rocket Lab Launch Complex": (-39.26085, 177.86586),

    # Iran
    "Shahroud Missile Range": (36.20092, 55.33366),
}

df["Launch Latitude"] = df["Launch Site"].map(lambda s: launch_site_coords.get(s, (pd.NA, pd.NA))[0])
df["Launch Longitude"] = df["Launch Site"].map(lambda s: launch_site_coords.get(s, (pd.NA, pd.NA))[1])

print("Rows missing Launch Latitude:", df["Launch Latitude"].isna().sum())

missing_sites = (
    df.loc[df["Launch Latitude"].isna(), "Launch Site"]
    .value_counts()
)

print("\nLaunch Sites missing coordinates (count):")
display(missing_sites)

Now that we have the coordinates, we can use the latiitude to calculate the distance (in kilometers) from the equator and to the nearest pole.

In [ ]:
KM_PER_DEGREE_LAT = 111.32

lat_numeric = pd.to_numeric(df["Launch Latitude"], errors="coerce")

df["Distance from Equator"] = (lat_numeric.abs() * KM_PER_DEGREE_LAT).round(1)
df["Distance from Pole"] = ((90 - lat_numeric.abs()) * KM_PER_DEGREE_LAT).round(1)

df[
    df["Launch Latitude"].notna()
][
    ["Launch Site", "Launch Latitude", "Distance from Equator", "Distance from Pole"]
].head(10)

While we're at it, let's also classify the launch location to fit nicely into a category.

In [ ]:
def classify_launch_location(lat):
    if pd.isna(lat):
        return "Unknown"

    abs_lat = abs(lat)

    if abs_lat < 20:
        return "Close to Equator"
    elif abs_lat < 50:
        return "Mid-Latitude"
    else:
        return "Close to Pole"


df["Launch Location Category"] = df["Launch Latitude"].apply(classify_launch_location)

df[
    df["Launch Latitude"].notna()
][
    ["Launch Site", "Launch Latitude", "Launch Location Category"]
].drop_duplicates().sort_values("Launch Latitude")



Finally, let's also record the hemisphere.

In [ ]:
def classify_hemisphere(lat):
    if pd.isna(lat):
        return "Unknown"
    if lat > 0:
        return "Northern"
    if lat < 0:
        return "Southern"
    return "Equator"

df["Hemisphere"] = df["Launch Latitude"].apply(classify_hemisphere)

df[
    df["Launch Latitude"].notna()
][["Launch Site", "Launch Latitude", "Distance from Equator", "Hemisphere"]].head(10)


### Finishing Touches
#### Country Names

Let's finish the off with a final cleanup of country names.

In [ ]:
df["UN Registry"] = (
    df["UN Registry"]
        .replace(r"^\s*$", pd.NA, regex=True)
        .fillna("Unknown")
)

df["Country of Operator"] = (
    df["Country of Operator"]
        .astype(str)
        .str.split("/")
        .str[0]
        .str.strip()
        .replace({"Multinational": "Unknown", "nan": pd.NA})
)

df["Country of Contractor"] = (
    df["Country of Contractor"]
        .astype(str)
        .str.split("/")
        .str[0]
        .str.strip()
        .replace({"Multinational": "Unknown", "nan": pd.NA})
)

df[
    ["Country of Operator", "Country of Contractor"]
].value_counts().head(10)


It looks like there are some inconsistencies in using country abbreviations in the different fields. Lets standardize this. We'll also create a boolean to countries that are operators, contractors, and have the registration.

In [ ]:
country_standardization = {
    "UK": "United Kingdom",
    "USA": "United States of America",
}

cols_to_standardize = ["UN Registry", "Country of Operator", "Country of Contractor"]

for col in cols_to_standardize:
    df[col] = (
        df[col]
            .astype("string")
            .str.strip()
            .replace(country_standardization)
    )

df["Is Country Fully Consistent"] = (
    (df["UN Registry"] == df["Country of Operator"]) &
    (df["Country of Operator"] == df["Country of Contractor"])
)

df["Is Operator/Contractor Country Match"] = (
    df["Country of Operator"] == df["Country of Contractor"]
)

df[
    [
        "Satellite",
        "UN Registry",
        "Country of Operator",
        "Country of Contractor",
        "Is Country Fully Consistent",
        "Is Operator/Contractor Country Match",
    ]
].head(10)

#### SpaceX
There are a lot of Starlink satellites in the sky. Let's create a boolean to flag these.

In [ ]:
df["Is SpaceX"] = df["Operator"] == "SpaceX"
df["Is SpaceX"].value_counts()

#### Finished
That's it! Let's get to analyzing.

In [ ]:
output_path = "data/ucs_satellite_cleaned.csv"

df.to_csv(output_path, index=False)

print(f"File written to: {output_path}")

![Loose Ends, Long Goodbyes Sat 2](img/lelg_sat_2_small.png)